In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=18,6
import seaborn as sns

In [ ]:
!pip install july
import july # for creating pretty heatmaps of daily data

# Outline

For comparing air quality around the globe during the three-year 2019-2021, one can download the Covid-19 Air Quality Worldwide csv datasets from the WAQI webpage https://aqicn.org/data-platform/covid19/
. The datasets cover about 380 major cities in the world, providing daily statistics (count, min, max, median, variance) of various air pollutant species (pm10, pm25 etc.) as well as meteorological data, based on several stations in each city, updated 3 times a day.

This notebook runs a quick summary and visualization based the WAQI data 2019-2021 for comparing any given subset of the cities.

# Read in and Get to Know the Data

In [ ]:
!curl --compressed -o waqi-covid-2021Q1.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2021Q1
!curl --compressed -o waqi-covid-2021Q2.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2021Q2
!curl --compressed -o waqi-covid-2021Q3.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2021Q3
!curl --compressed -o waqi-covid-2021Q4.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2021Q4
!curl --compressed -o waqi-covid-2020Q1.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2020Q1
!curl --compressed -o waqi-covid-2020Q2.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2020Q2
!curl --compressed -o waqi-covid-2020Q3.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2020Q3
!curl --compressed -o waqi-covid-2020Q4.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2020Q4
!curl --compressed -o waqi-covid-2019Q1.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2019Q1
!curl --compressed -o waqi-covid-2019Q2.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2019Q2
!curl --compressed -o waqi-covid-2019Q3.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2019Q3
!curl --compressed -o waqi-covid-2019Q4.csv https://aqicn.org/data-platform/covid19/report/27298-25cc14ef/2019Q4

In [ ]:
li = []

period_names = ['2019Q1', '2019Q2', '2019Q3', '2019Q4', 
                '2020Q1', '2020Q2', '2020Q3', '2020Q4', 
                '2021Q1', '2021Q2', '2021Q3', '2021Q4']

for period in period_names:
    df = pd.read_csv(#"/kaggle/working/waqi-covid-%s.csv"%(period), 
                     "/kaggle/input/air-quality-comparison-2019-2021/waqi-covid-%s.csv"%(period), 
                     header=4,
                     parse_dates=['Date'],
                     infer_datetime_format=True)
    li.append(df)

waqi_data = pd.concat(li, axis=0)

In [ ]:
waqi_data

In [ ]:
# check for potential name variation/confusion of any given subset of cities

cities_partial_name = 'jerus|london|new|shang|toky|san jo|paris|perth|tel aviv|osl|bos'
cities_pool = waqi_data.loc[waqi_data.City.str.lower().str.contains(cities_partial_name)][['City','Country']].value_counts()
cities_pool

In [ ]:
# finalize the list of selected cities
cities_selected = cities_pool.index.values[[0,1,2,3,4,5,7,8,11,12]]
cities_selected

In [ ]:
# different pollutant species of air quality, and meterological measurements
waqi_data.Specie.value_counts()

# Explore & Visualize the Data

First, to see what cities have the cleanest air and etc., we provide the function below.

In [ ]:
# return top n cities in the globe after sorting by average statistic of some daily air quality measurement specie
def top_cities(n = 10, data = waqi_data, statistic = 'median', measurement_specie = 'pm25'
                       , start = '2019-01-01', end = '2022-01-01', ascending = False):
    
    analysis_data = data.loc[data.Specie.isin([measurement_specie])]\
                                   .groupby(['City', 'Country', 'Date'])
    
    y = analysis_data[statistic].mean().unstack(['City', 'Country']).loc[start:end]
    
    if ascending == False:
        z = y.mean().sort_values(ascending=False).head(n)
    else:
        z = y.mean().sort_values().head(n)
    
    display(z)
    
    return z.index.values

In [ ]:
# 10 cities of the lowest average daily median pm25 during 2019-2021
z = top_cities(n = 10, data = waqi_data, statistic = 'median', measurement_specie = 'pm25'
                       , start = '2019-01-01', end = '2022-01-01', ascending = True)

display(z)

In [ ]:
# 10 cities of the highest average daily median pm25 during 2019-2021
z = top_cities(n = 10, data = waqi_data, statistic = 'median', measurement_specie = 'pm25'
                       , start = '2019-01-01', end = '2022-01-01', ascending = False)

display(z)

Second, a function to visualize the air quality on a calendar.

In [ ]:
# plot calendar heatmap for chosen cities, statistic and measurement specie
def calendar_heatmap(data = waqi_data, cities = cities_selected, statistic = 'median', measurement_specie = 'pm25'
                       , start = '2019-01-01', end = '2022-01-01'):
    
    analysis_data = data.loc[[x in list(cities) for x in list(zip(data['City'], data['Country']))]\
                                   & data.Specie.isin([measurement_specie])]\
                                   .groupby(['City', 'Country', 'Date'])
    
    y = analysis_data[statistic].mean().unstack(['City', 'Country']).loc[start:end]
    
    for city,country in y.columns:
        plt.figure()
        july.heatmap(y.index, y[(city,country)], title='%s %s DAILY %s %s'% (city, country, statistic.upper(), measurement_specie.upper()), cmap="golden", colorbar=True)
        plt.show()
    

In [ ]:
calendar_heatmap()

Next we provide two more summary functions, one for overall comparison, the other for pairwise comparison. They will print out some plots and summaries based on selected measurements/statistics.

In [ ]:
# overall comparison summary function
def overall_comparison_summary(data = waqi_data, cities = cities_selected, statistic = 'median', measurement_specie = 'pm25'
                       , start = '2019-01-01', end = '2022-01-01'):
    
    analysis_data = data.loc[[x in list(cities) for x in list(zip(data['City'], data['Country']))]\
                                   & data.Specie.isin([measurement_specie])]\
                                   .groupby(['City', 'Country', 'Date'])
    
    y = analysis_data[statistic].mean().unstack(['City', 'Country']).loc[start:end]
    
    # average rank of cities by chosen statistic of daily measurment
    print('\nAverage rank of cities by DAILY %s %s:\n' % (statistic.upper(), measurement_specie.upper()))
    display(y.rank(axis=1).mean())
    
    # average chosen statistic of daily measurment (unweighted)
    print('\n\nAverage DAILY %s %s:\n' % (statistic.upper(), measurement_specie.upper()))
    display(y.mean())
    
    # average chosen statistic of daily measurment (weighted by daily measurement count)
    print('\n\nAverage DAILY %s %s ((weighted by daily measurement count)):\n' % (statistic.upper(), measurement_specie.upper()))
    w = analysis_data['count'].mean().unstack(['City','Country']).loc[start:end]
    display((w * y).sum() / w.sum())
    
    # distribution (boxen plot) of chosen statistic of daily measurment
    print('\n\nDistribution (boxen plot) of DAILY %s %s:' % (statistic.upper(), measurement_specie.upper()))
    g = sns.catplot(data=y, kind='boxen')\
        .set_axis_labels('City', 'DAILY %s %s'% (statistic.upper(), measurement_specie.upper()))
    g.set_xticklabels(rotation=42)
    print('\n\n')
    
    # time series plot of Daily Median PM2.5
    y.plot(title='Time series of DAILY %s %s:' % (statistic.upper(), measurement_specie.upper()), ylabel='%s %s:' % (statistic.upper(), measurement_specie.upper()))
    
    return None

In [ ]:
# pairwise comparison summary function
def pairwise_comparison_summary(data = waqi_data, cities = cities_selected, statistic = 'median', measurement_specie = 'pm25'
                       , start = '2019-01-01', end = '2022-01-01'):
    
    analysis_data = data.loc[[x in list(cities) for x in list(zip(data['City'], data['Country']))]\
                                   & data.Specie.isin([measurement_specie])]\
                                   .groupby(['City', 'Country', 'Date'])
    
    y = analysis_data[statistic].mean().unstack(['City', 'Country']).loc[start:end]
    
    d = y[cities[0]] - y[cities[1]]
    
    #
    print('\nPercentage of time when DAILY %s %s in %s is greater than %s: %.2f\n' % (statistic.upper(), measurement_specie.upper(), *cities, (d>0).mean()))
    print('Percentage of time when DAILY %s %s in %s is equal %s: %.2f\n' % (statistic.upper(), measurement_specie.upper(), *cities, (d==0).mean()))
    print('Percentage of time when DAILY %s %s in %s is less than %s: %.2f\n' % (statistic.upper(), measurement_specie.upper(), *cities, (d<0).mean()))


    
    # distribution (boxen plot) of difference in chosen statistic of daily measurment
    print('\nDistribution (boxen plot) of difference in DAILY %s %s between %s and %s:' % (statistic.upper(), measurement_specie.upper(), cities[0], cities[1]))
    g = sns.catplot(data=d.to_frame(), kind='boxen')\
        .set_axis_labels('%s - %s' % (cities[0], cities[1]), 'difference in %s' % measurement_specie.upper())
    g.set_xticklabels(rotation=30)

    
    # time series plot of Daily Median PM2.5
    plt.figure()
    ax = d.plot(xlabel=' ', ylabel='difference in %s' % measurement_specie.upper(), title='difference in DAILY %s %s between %s and %s' % (statistic.upper(), measurement_specie.upper(), *cities))
    ax.axhline(y=d.mean(), xmin=-1, xmax=1, color='r', linestyle='--', lw=2)
    ax.axhline(y=0, xmin=-1, xmax=1, color='k', linestyle='-', lw=1)
    
    return None

## Daily Median PM2.5

In [ ]:
overall_comparison_summary()

## Daily Min PM2.5

In [ ]:
overall_comparison_summary(statistic='min')

## Daily Max PM2.5

In [ ]:
overall_comparison_summary(statistic='max')

## Daily Variance PM2.5

In [ ]:
overall_comparison_summary(statistic='variance')

## Pairwise Comparison

In [ ]:
overall_comparison_summary(cities = [('San Jose', 'US'), ('Tokyo', 'JP')], statistic = 'median')

In [ ]:
calendar_heatmap(cities = [('San Jose', 'US'), ('Tokyo', 'JP')], statistic = 'median')

In [ ]:
pairwise_comparison_summary(cities = [('San Jose', 'US'), ('Tokyo', 'JP')], statistic = 'median', measurement_specie = 'pm25')

# Conclusion

As shown above, the comparisons of air quality between the cities across the globe actually exibit a somewhat more complicated picture than people often thought. For example, in terms of Daily Min PM2.5, people who live in Tokyo seem to enjoy cleaner air than those in San Jose. But in terms of Daily Median PM2.5, we probably can say the opposite. So what air quality pollutant species and statistics are most important to you? And for which cities are the comparisons of most interest to you? Experiment with the data and code above and have fun.